
# Лекция 10
## Объектно-ориентированное программирование (продолжение)


__Автор: Сергей Вячеславович Макрушин__ e-mail: SVMakrushin@fa.ru 

Финансовый универсиет, 2021 г.  

v 0.7 15.02.2022

## Разделы: <a class="anchor" id="разделы"></a>

* [Методы классов и статические переменные и методы](#методы-классов)    
* [Управление доступом к атрибутам класса](#управление-доступом)    
* [Динамические операции с атрибутами и интроспекция](#интроспекция)    
* [Специальные методы](#специальные)                 
-

* [к оглавлению](#разделы)

In [1]:
# загружаем стиль для оформления презентации
from IPython.display import HTML
from urllib.request import urlopen
html = urlopen("file:./lec_v1.css")
HTML(html.read().decode('utf-8'))


## Методы классов и статические переменные и методы <a class="anchor" id="методы-классов"></a>
* [к оглавлению](#разделы)

In [2]:
class Ship(object):
    next_index = 0  # переменная класса (статическая переменная)
    
    @classmethod
    def generate_next_index(cls): # в classmethod первый обязательный параметр: cls - переменная, ссылающаяся на КЛАСС
        index = cls.next_index
        cls.next_index += 1
        return index
    
    def __init__(self):
        self.index = Ship.generate_next_index()
        
    @staticmethod
    def is_from_same_epoch(sh1, sh2): # не имеет доступа ни к объекту ни к классу
        return abs(sh1.index - sh2.index) < 10

In [3]:
s1 = Ship()
s1.index

0

In [4]:
fleet = [Ship() for _ in range(15)]

In [5]:
for sh in fleet:
    print(sh.index)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [6]:
Ship.next_index # доступ к переменной класса через имя класса

16

In [7]:
s1.next_index # доступ к переменной класса через объект

16

In [8]:
print([s.next_index for s in fleet])

[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16]


In [9]:
fleet[0].next_index = 100 # приводит не к изменению в переменной класса, а к появлнию нового атрибута у данного объкта!
print([s.next_index for s in fleet], Ship.next_index)

[100, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16] 16


In [10]:
Ship.next_index = 50 # изменяем значение переменной класса
print([s.next_index for s in fleet], Ship.next_index)

[100, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50] 50


In [11]:
Ship.generate_next_index() # доступ к методу класса через имя класса

50

In [12]:
Ship.next_index

51

Статический метод

In [13]:
Ship.is_from_same_epoch(fleet[0], fleet[-1])

False

In [14]:
s1.is_from_same_epoch(s1, fleet[0])

True

In [15]:
for s in fleet:
    print(s, )

## Управление доступом к атрибутам класса <a class="anchor" id="управление-доступом"></a>
* [к оглавлению](#разделы)

In [17]:
cc_1.load, cc_1.max_load, cc_1.is_overloaded()

NameError: name 'cc_1' is not defined

In [18]:
# нарушаем правила загрузки грузового автомобиля:
cc_1.load = 12
cc_1.is_overloaded()

NameError: name 'cc_1' is not defined

In [ ]:
cc_1.max_load = 20

In [ ]:
cc_1.is_overloaded()

In [ ]:
# Класс CargoCar с контролем доступа к значениям загрузки и максимального предела загрузки 
class CargoCar2(Car): 
    def __init__(self, x, max_load, load):
        self.x = x
        self.__max_load = max_load
        self.__load = load
        assert not(self.is_overloaded()), 'При создании автомобиля превышено ограничение загрузки!'
    
    def is_overloaded(self):
        return self.__load > self.__max_load
    
    def get_load(self):
        return self.__load
    
    def set_load(self, load): # проверка при изменении значения
        assert load < self.__max_load, "Превышен предел загрузки!"
        self.__load = load
        
    def get_max_load(self): # для max_load есть только возможность получения значения
        return self.__max_load

In [ ]:
сс2_1 = CargoCar2(5.0, 10, 11)

In [ ]:
cc2_2 = CargoCar2(5.0, 10, 9)

In [ ]:
cc2_2.__load #приватная переменная защищена от доступа извне класса

In [ ]:
cc2_2.get_load()

In [ ]:
cc2_2.set_load(8)
cc2_2.get_load()

In [ ]:
cc2_2.set_load(11)

In [ ]:
# Класс CargoCar с контролем доступа к значениям, выполненным в стиле Python
class CargoCar3(Car): 
    def __init__(self, x, max_load, load):
        self.x = x
        self.__max_load = max_load
        self.__load = load
        assert not(self.is_overloaded()), 'При создании автомобиля превышено ограничение загрузки!'
    
    def is_overloaded(self):
        return self.__load > self.__max_load
    
    @property # Декоратор функции, оформляющий функцию как функцию доступа
    def load(self):
        return self.__load
    
    @load.setter # Декоратор функции, оформляющий функцию как функцию-сеттер
    def load(self, val): # проверка при изменении значения
        assert val < self.__max_load, "Превышен предел загрузки!"
        self.__load = val
        
    # при необходимости, есть декоратор вида: @load.deletter
        
    @property 
    def max_load(self): # для max_load есть только возможность получения значения
        return self.__max_load

In [ ]:
cc3_1 = CargoCar3(5.0, 10, 9)

In [ ]:
cc3_1.__load

In [ ]:
cc3_1.load

In [ ]:
cc3_1.load = 8
cc3_1.load

In [ ]:
cc3_1.load = 11

In [ ]:
cc3_1.max_load

In [ ]:
cc3_1.max_load = 7

## Динамические операции с атрибутами и интроспекция <a class="anchor" id="интроспекция"></a>
* [к оглавлению](#разделы)

In [ ]:
# создаем объект, при создании передаем параметр конструктора:
ob_1 = Car(3.1)
ob_2 = Car(4.1)
ob_3 = CargoCar3(5.0, 10, 9)
my_objects = [ob_1, ob_2, ob_3]

In [ ]:
ob_1.x

In [ ]:
ob_1.length

In [ ]:
# присваиваем объекту значение для нового атрибута
ob_1.length = 11

In [ ]:
ob_1.length

In [ ]:
# у других объектов этого же типа данный атрибут отсутствует:
ob_2.length

In [ ]:
# атрибут у объекта можно не только создать, но и удалить:
del ob_1.length

In [ ]:
ob_1.length

Получать значения атрибутов, задвать значения атрибутов и удалять их можно по их имени, хранящемуся в виде строки при помощи встроенных функций: 

getattr()

setattr()

delattr()

In [ ]:
new_attr = 'number'
for i, o in enumerate(my_objects):
    setattr(o, new_attr, i)

In [ ]:
ob_2.number

In [ ]:
getattr(ob_2, new_attr)

In [ ]:
getattr?

Данные функции позволяют обращаться к атрибутам, имена которых заранее неизвестны. Это особенно важно для реализации интроспекции. Интроспекция (type introspection) в программировании — возможность в объектно-ориентированных языках определить тип и структуру объекта во время выполнения программы. Эта возможность присуща языкам, позволяющих манипулировать типами объектов как объектами первого класса (first class citizens). 

In [ ]:
dir(ob_3)

In [ ]:
?dir

In [ ]:
# получаем значения и тип всех незащищенных переменных объекта:
for v_name in dir(ob_3):
    if v_name[0] == '_':
        continue
    attr = getattr(ob_3, v_name)
    print(v_name, attr, type(attr))

In [ ]:
# получаем значения и тип всех незащищенных переменных объекта:
import types

for v_name in dir(ob_3):
    if v_name[0] == '_':
        continue
    attr = getattr(ob_3, v_name)
    attr_t = type(attr)
    if attr_t is types.MethodType:
        print(v_name, '(method)', attr_t)
    else:
        print(v_name, attr, attr_t)

In [ ]:
# Возвращает только аттрибуты объекта для которых можно и получить и задать значения:
vars(ob_3)

In [ ]:
# наличие атрибута можно проверить с помощью функции hasattr():
for o in my_objects:
    if hasattr(o, 'load'):
        print(o.number, o.load)

Встроенные функции для выполнения задач объектно-ориентированного программирования:

http://python-reference.readthedocs.io/en/latest/docs/functions/#object-oriented-functions


https://docs.python.org/3/library/functions.html


## Специальные методы <a class="anchor" id="специальные"></a>
* [к оглавлению](#разделы)

\__repr\__(self) и \__str\__(self) - служат для nреобразования объекта в строку. Метод \__repr\__()  вызывается nри выводе в интерактивной оболочке, а также nри исnользовании функции repr(). Метод \__str\__() вызывается nри выводе с nомощью функции print(), а также nри исnользовании функции str(). Если метод \__str\__() отсутствует, то будет вызван метод \__repr\__(). В качестве значения методы \__repr\__() и \__str\__() должны возвращать строку. Причем, значение возвращаемое \__repr\__() по возможности должно возврващать строку имеющую вид конструктора аналогичного объекта. Т.е. должно быть истинно выражание: eval(repr(obj)) == obj. 

In [21]:
eval('[11, 22]+[33]')

[11, 22, 33]

In [22]:
for s in fleet:
    print(s)

In [23]:
class ShipS(Ship):
    def __str__(self):
        return f'Ship with index {self.index}'

In [ ]:
fleet2 = [ShipS() for _ in range(5)]
for s in fleet2:
    print(s)

Когда объект используется в строке формата, вызывается метод \__format\__(self, format_spec) объекта с самим объектом и  спецификацией формата в виде аргументов. Метод возвращает строку с экземпляром, отформатированным  соответствующим образом.

Специальные методы для поддержки преобразования типов:

* \__bооl__(self) - вызывается при исnользовании функции bool() 
* \__int__(self) - вызывается nри преобразовании объекта в целое число с nомощью функции int ()
* \__float__(self) - вызывается nри nреобразовании объекта в вещественное число с nомощью функции float (); 
* \__complex__(self) -вызывается nри исnользовании функции complex ()

Специальные мтоды для поддержки операций сравнения:

* х == у - равно - х.\__еq\__(у) 
* х != у - неравно - х.\__nе\__(у)
* х < у - меньше - х.\__lt\__(y) 
* х > y - бoльwe - x.\__gt\__(y)
* х <= у - меньшеилиравно - х.\__lе\__(у)
* х >= у - больше или равно - x.\__ge\__(y)
* у in х - nроверка на вхождение - х.\__contains__(у)

Интерпретатор Python будет автоматически подставлять метод \__ne\__() (not equal - не равно), реализующий действие оператора неравенства (!=), если в классе присутствует реализация метода \__eq\__(), но отсутствует реализация метода \__ne__() . 


По умолчанию экземпляры наших собственных классов поддерживают оператор == (который всегда возвращает False) и являются хешируемыми (поэтому они могут использоваться в качестве ключей словаря или добавляться в множества). Но если реализовать специальный метод \__eq\__(), выполняющий корректную проверку на равенство, экземпляры перестанут быть хешируемыми. Это можно исправить, реализовав специальный метод \__hash\__(). Язык Python предоставляет функцию хеширования строк, чисел, фиксированных множеств и других классов. 

Специальный метод \__del\__(self) вызывается при уничтожении объекта - по крайней мере в теории. На практике метод \__del\__() может не вызываться никогда, даже при завершении программы. 

### Материалы для подготовки к следующей лекции:

Прохоренок: Глава 11 "Пользовательские функции"

Саммерфильд: Глава 8 "Улучшенные приемы программирования" (разделы: Улучшенные приемы процедурного программирования; Функциональное программирование